In [ ]:
# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 48

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
image_idx = [0,2,4,6,7,8,10,12,15,16,18,20,22,23,24,26,28,29]
idx_len = len(image_idx)
print(idx_len)

image_height = 96
image_width = 96


18


In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = image_idx #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # Reshaping images and trimming it first from corners if shape has 160 width
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(image_height,image_width)).astype(np.float32)
                    else:
                        image = imresize(image,(image_height,image_width)).astype(np.float32)
                        
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    # Doing max normalization rather than dividing just by 255.
                    batch_data[folder,idx,:,:,0]  = (image[:,:,0]) / (image[:,:,0]).max()
                    batch_data[folder,idx,:,:,1]  = (image[:,:,1]) / (image[:,:,1]).max()
                    batch_data[folder,idx,:,:,2]  = (image[:,:,2]) / (image[:,:,2]).max()
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_sequemce_len = 0
        if((len(t)%batch_size)!=0):
            rem_sequemce_len = len(t)%batch_size
        
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((rem_sequemce_len,18,image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem_sequemce_len,5)) # batch_labels is the one hot representation of the output

            for folder in range(rem_sequemce_len): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(image_height,image_width)).astype(np.float32)
                    else:
                        image = imresize(image,(image_height,image_width)).astype(np.float32)
                    #image = imresize(image,(120,120,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #image = image_augmentation(image)
                    
                    batch_data[folder,idx,:,:,0]  = (image[:,:,0]) / (image[:,:,0]).max()
                    batch_data[folder,idx,:,:,1]  = (image[:,:,1]) / (image[:,:,1]).max()
                    batch_data[folder,idx,:,:,2]  = (image[:,:,2]) / (image[:,:,2]).max()
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here

Input_shape = (idx_len, image_height, image_width, 3)
filter_size = (3,3,3)

model = Sequential()
model.add(Conv3D(16,filter_size , strides=(1,1,1), padding='same', input_shape=Input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(32,filter_size, strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(64, filter_size, strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = tf.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 96, 96, 16)    1312      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 96, 96, 16)    64        
_________________________________________________________________
activation (Activation)      (None, 18, 96, 96, 16)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 48, 96, 16)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 48, 96, 32)     13856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 48, 96, 32)     128       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 48, 96, 32)     0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

model_name = 'model_4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss',
                                    mode = 'min',
                                    save_best_only = True,
                                    verbose = 1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3, verbose = 1)
        
#early_stop = EarlyStopping(monitor = "loss", patience=9, restore_best_weights =True, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
hist1 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 48
Epoch 1/50
14/14 [==============================] - ETA: 0s - loss: 1.5987 - categorical_accuracy: 0.4072Source path =  /home/datasets/Project_data/val ; batch size = 48

Epoch 00001: val_loss improved from inf to 1240.78430, saving model to model_4_2021-11-2412_27_45.441123/model-00001-1.59872-0.40724-1240.78430-0.23000.h5
14/14 [==============================] - 79s 6s/step - loss: 1.5987 - categorical_accuracy: 0.4072 - val_loss: 1240.7843 - val_categorical_accuracy: 0.2300
Epoch 2/50
14/14 [==============================] - ETA: 0s - loss: 1.1602 - categorical_accuracy: 0.5230
Epoch 00002: val_loss improved from 1240.78430 to 867.39252, saving model to model_4_2021-11-2412_27_45.441123/model-00002-1.16020-0.52305-867.39252-0.23000.h5
14/14 [==============================] - 70s 5s/step - loss: 1.1602 - categorical_accuracy: 0.5230 - val_loss: 867.3925 - val_categorical_accuracy: 0.2300
Epoch 3/50
14/14 [============